# Executive Summary

## Notebook Workflow

### Preprocessing Steps and Features Engineering (Across Datasets)

- **Removal of `offer_decline_count` on Model:**
  - Analysis from the `train.describe()` function reveals that `offer_decline_count` has a mean and standard deviation close to zero. This suggests that this variable is unlikely to influence the model significantly, leading to its exclusion from further analysis.

- **Removal of Missing Values:**
  - Upon preprocessing, missing values were identified in four columns, each accounting for approximately 4.15% of the dataset. To address this issue, all missing values were dropped from the training dataset, preparing it for subsequent processing.

- **Adding the Duration of each Session**
  - Duration in minutes was computed for each session, representing the time from entering the website to session end, spanning multiple rows in the dataset.

- **Basic Feature Extraction from `event_time` Column: Extracting the year, month, day, hour, and day name from session_start_time**
  - Basic feature extraction was performed on the `event_time` column to derive additional insights (more time data).

- **Adding a Day Range based on the New 'day' column:**
  - An additional column named `day_range` was created to categorize sessions into thirds of the month, providing further granularity in temporal analysis.

- **Conversion of Columns and Removal of Redundant Ones:**
  - Newly added columns were converted to string format, and redundant columns such as `session_start_time`, `session_expiry_time`, and `event_time` were dropped, as they were now considered redundant.

  - **Normalization of Numerical Columns:**
  - Numerical columns were normalized to ensure uniform scales across features, preventing certain variables from disproportionately influencing the model.

- **One-Hot Encoding of Categorical Columns:**
  - Categorical columns were one-hot encoded to convert them into a numerical format suitable for machine learning algorithms, allowing the model to effectively interpret and utilize categorical data.

### Model Fit and Test on the Kaggle Dataset: Neural Network Model

- **Input Layer**: 
  - 64 neurons with ReLU activation.
  - Input shape determined by `X_train.shape[1]`.

- **Hidden Layer**: 
  - 35 neurons with ReLU activation.

- **Output Layer**: 
  - Single neuron with linear activation.

- **Optimizer**: Adam optimizer.

- **Loss Function**: Mean Squared Error (MSE).

- **Epochs**: 116 epochs.

- **Batch Size**: 850.

- **RMSE Calculation**: 
  - Train RMSE computed using `mean_squared_error`.
  - Lower RMSE indicates better model performance.

- **Predictions**:
- Model used to predict on `data_kaggle_test`.
- Predictions stored in `test_predictions_df`.

## TRIAL Steps (Removed from the Notebook Workflow)

### Trial Steps for Preprocessing and Feature Engineering

**Filter the dataset to keep only rows where 'page_load' is equal to 'checkout'**
  - In an attempt to reduce the dataset size and considering the fact that we are trying to predict the total money spent by a customer (it is recorded at chekcout). I tried to remove all the rows where the 'page_type' column was different than 'Checkout', resulting in a dataset size of about 150000 rows.
  - However, I removed this step and replaced it with a subsetting technique that will stratify based on the 'page_type' column, specifically the 'Checkout' category. This way, my subset will have the same percentage of 'Checkout' as the original Dataset

**Outlier Detection in Numerical Columns:**
  - Notably, the columns `cart_total` and `last_reward_value` exhibit the highest percentage of outliers.-
  - However, I decided to remove this step because the number of outliers in the columns is large (around 10% for some columns). This led to overfitting as the model was not being exposed to a lot of observations (the outliers removed)

- **Treatment of Extreme Values in Target Variable:**
  - Following exploratory data analysis (EDA), extreme values in the target variable were identified, with the mean close to 140 and maximum value at 197,000.
  - However, I decided to keep these extreme values for the same reasons I kept the outliers in the dataset.

**Conversion of Pixel Count to Categorical Variable:**
  - The pixel count of devices was deemed too precise and not sufficiently informative. Consequently, this column was converted into a categorical variable with two types: 'Mobile phones' and 'Laptops'. 
  - However, I decided to remove this step because it was modifying the number of rows in the Kaggle dataset, which is not permitted when submitting to the competition

**Expansion of the 'cart_data' column:**
- Convert the string representation of dictionaries into actual dictionaries
- Explode the cart_data column to separate rows
- Expand the cart_data dictionary into separate columns
- Replace the prices with the average price per category
- Add a 'rank' column that ranks the categories based on their count
- However, I decided to remove this step because it was adding new rows (in case a client ordered more than one product),which is not permitted when submitting to the competition. The 'cart_data' column was later dropped as it would be considered as a categorical variable in my model.



### Trial Steps for Model Selection

- **Linear Regression Model:**
  - The RMSE on the train was relatively low
  - However, the RMSE on test was extrmely high (in the millions).

- **KNN Model:**
  - The RMSE on the test is higher than the mean indicating a very poor model.
  - This suggests that the KNN model is overfitting the training data.

- **Random Forest Model:**
  - The Random Forest model exhibits overfitting, as seen in the substantial gap between training and test RMSE values.
  - Despite attempting to mitigate overfitting with Random Forest, the model still performs poorly, with an RMSE significantly higher than the target mean.
  - Further refinement in feature selection and preprocessing steps may be necessary to address this issue.
  - A Grid Search was done for the random forest model and it resulted in low RMSE scores for both train and test set. However, the model was taking a lot to run and it was performing poorly on the Kaggle test set

- **Decision Tree Model:**
  - The Decision Tree model shows severe overfitting, with a large discrepancy between training and test RMSE values.
  - This indicates that the model is not generalizing well to unseen data.

- **Gradient Boosting Model:**
  - The Gradient Boosting model, although showing some improvement, still yields relatively high RMSE values on training data and testing data
  - This suggests that the Gradient Boosting model is also overfitting the training data.

- **The GradientBoostingRegressor:**
  -The RMSE for the Gradient Boosting Models was low, however it was also overfitting

**The main reason why these models were not used as final models is because of overfitting, which resulted in poor performance on the kaggle competition**

## Important Notes and Challenges

- It is important to note the the random forest model was performing well as it is an ensemble model (good for overfitting). However, the neural network model was constantly topping the random forest on the Kaggle competition.

- Dataset was too large when fitting the models. This was especially a challenge for grid searchs. This is why I decided to subset the data.

- Most traditional machine learning models were overfitting, which led me to venture into neural network models

- **It is important to note that the prediciton file generated from the neural network model will not be the same as the one on Kaggle. This is due to the fact that my subsetting technique (stratify on checkout) did not include any random state position, which will result in a different subset at each run. In addition, the neural network model used will yield different results after each run due to many factors such as: Random Initialization, Mini-Batch Gradient Descent, Randomness in Regularization.**

# Importing the Packages

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import ast

## Reading the Dataset

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
#data= pd.read_csv('/content/drive/MyDrive/data.zip')

In [4]:
data = pd.read_csv('C:\\Users\\Lenovo\\OneDrive\\01_Education\\02_AUB\\02_Spring2024\\MSBA315-Introduction to Python for Machine Learning\\Assignments\\Assignment5\\AssignmentFolder\\data\\data.csv')

## Creating a Subset of the data by Stratifying on the 'page_type' Column

In [5]:
# Assuming your dataset is stored in a pandas DataFrame called 'df'
# 'stratify_column' refers to the column you want to stratify the sampling on

# Calculate the proportions of each category in the stratification column
strata_props = data['page_type'].value_counts(normalize=True)

# Determine the number of samples to take from each stratum
num_samples_per_stratum = (strata_props * 300000).round().astype(int)

# Perform stratified sampling for each category
stratified_samples = []
for category, num_samples in num_samples_per_stratum.items():
    stratum_samples = data[data['page_type'] == category].sample(n=num_samples, random_state=42)
    stratified_samples.append(stratum_samples)

# Concatenate the stratified samples into a single DataFrame
data = pd.concat(stratified_samples)

# Reset index if needed
data.reset_index(drop=True, inplace=True)

## Converting Time Columns to DateTime Objects

In [6]:
# Convert object columns to datetime objects
data['session_start_time'] = pd.to_datetime(data['session_start_time'])
data['session_expiry_time'] = pd.to_datetime(data['session_expiry_time'])
data['event_time'] = pd.to_datetime(data['event_time'])
data['event_time_zone'] = data['event_time_zone'].astype('str')

## Splitting the Dataset

In [7]:
train, valid = train_test_split(data, test_size=0.2, shuffle=True, random_state=42)

# Preprocessing and Feature Engineering for the Train Set

## Removing Zero or Near Zero Columns

In [8]:
#mean is approx to 0 in 'offer_decline_count"
train = train.drop('offer_decline_count', axis=1)
train=train.reset_index(drop=True)

## Droping Null values

In [9]:
# Assuming 'data' is your DataFrame
train.dropna(thresh=data.shape[1]-1, inplace=True)

# Reset the index after dropping rows
train.reset_index(drop=True, inplace=True)
train=train.reset_index(drop=True)

## TRIAL: Outlier Removal

In [10]:
# Define the numerical variables
# numerical_variables = ['cart_quantity', 'cart_total', 'last_reward_value', 'last_spend_value', 'user_screen_size']

# Loop over each numerical feature
# for feature in numerical_variables:
#     Calculate quartiles and IQR for the current feature
#     Q1 = train[feature].quantile(0.25)
#     Q3 = train[feature].quantile(0.75)
#     IQR = Q3 - Q1

#     Define lower and upper bounds for outliers
#     lower_bound = Q1 - 1.5 * IQR
#     upper_bound = Q3 + 1.5 * IQR

#     Filter out rows with outliers in the current feature
#     outliers_mask = (train[feature] < lower_bound) | (train[feature] > upper_bound)
#     train = train.drop(train[outliers_mask].index)

# Reset index after dropping rows
# train.reset_index(drop=True,inplace=True)

In [11]:
# # Define the numerical variables
# numerical_variables = ['cart_quantity', 'cart_total', 'last_reward_value', 'last_spend_value', 'user_screen_size']

# # Loop over each numerical feature
# for feature in numerical_variables:
#     # Calculate quartiles and IQR for the current feature
#     Q1 = train[feature].quantile(0.25)
#     Q3 = train[feature].quantile(0.75)
#     IQR = Q3 - Q1

#     # Define lower and upper bounds for outliers
#     lower_bound = Q1 - 1.5 * IQR
#     upper_bound = Q3 + 1.5 * IQR

#     # Replace outliers with minimum or maximum value
#     min_val = train[feature].min()
#     max_val = train[feature].max()
#     train.loc[train[feature] < lower_bound, feature] = min_val
#     train.loc[train[feature] > upper_bound, feature] = max_val
    
# # Reset index after modifying the DataFrame
# train.reset_index(drop=True, inplace=True)

## Adding the Duration of each Session

In [12]:
# Group by 'session_id' and calculate the duration
session_durations = train.groupby('session_id').agg({
    'session_start_time': 'min',
    'session_expiry_time': 'max'
}).reset_index()

# Calculate the duration as the difference between the max expiry time and the min start time
session_durations['duration'] = session_durations['session_expiry_time'] - session_durations['session_start_time']

# Convert duration to a more convenient unit: minutes
session_durations['duration_minutes'] = session_durations['duration'].dt.total_seconds() / 60  # Convert seconds to minutes

# Round the duration_minutes column to 2 decimal places (nearest 0.01)
session_durations['duration_minutes'] = session_durations['duration_minutes'].round(2)

# Now merge this back into the original dataframe
train = train.merge(session_durations[['session_id', 'duration_minutes']], on='session_id', how='left')

# Verify the results
print(train[['session_id', 'session_start_time', 'session_expiry_time', 'duration_minutes']].head())

                             session_id               session_start_time  \
0  65394f9c-00e3-405b-8166-4df6d263927b 2020-01-29 03:55:54.348000+00:00   
1  8213705e-4fad-46ef-aaf9-5a057399a621 2019-10-09 05:24:04.238000+00:00   
2  12e86b40-60bb-48e5-b57c-bc50ad2e1ecc 2020-02-08 15:15:10.977000+00:00   
3  92c9b95e-dd23-43f3-9646-40b8ed555599 2020-02-03 15:23:45.041000+00:00   
4  a4dc3781-f2d9-4365-950e-d35b7b6a1897 2019-11-25 14:06:50.778000+00:00   

               session_expiry_time  duration_minutes  
0 2020-01-29 04:38:47.193000+00:00             77.78  
1 2019-10-09 06:33:48.166000+00:00             69.73  
2 2020-02-08 15:49:55.965000+00:00             34.75  
3 2020-02-03 16:14:48.806000+00:00             63.15  
4 2019-11-25 14:53:52.912000+00:00             66.75  


## Extracting the year, month, day, hour, and day name from session_start_time

In [13]:
# Extract year, month, day, hour, and day name from session_start_time before normalizing
train['year'] = train['event_time'].dt.year
train['month'] = train['event_time'].dt.month
train['day'] = train['event_time'].dt.day
train['hour'] = train['event_time'].dt.hour  # Extract hour before normalizing
train['day_of_week'] = train['event_time'].dt.day_name()

# Now you have separate columns for year, month, day, hour, and day_of_week along with a combined string if needed
print(train[['year', 'month', 'day', 'hour', 'day_of_week']].head())

   year  month  day  hour day_of_week
0  2020      1   29     4   Wednesday
1  2019     10    9     6   Wednesday
2  2020      2    8    15    Saturday
3  2020      2    3    15      Monday
4  2019     11   25    14      Monday


## Adding a Day Range based on the New 'day' column

In [14]:
# Function to categorize days with updated labels
def categorize_day(day):
    if 1 <= day <= 10:
        return 1  # Updated label for days 1-10
    elif 11 <= day <= 21:
        return 2  # Updated label for days 11-21
    else:
        return 3  # Updated label for days 22-31

# Apply the updated function to the 'day' column to create a new 'day_range' column with numerical labels
train['day_range'] = train['day'].apply(categorize_day)

# Display the first few rows to verify the new column
print(train[['year', 'month', 'day', 'day_range', 'hour', 'day_of_week']].head())

   year  month  day  day_range  hour day_of_week
0  2020      1   29          3     4   Wednesday
1  2019     10    9          1     6   Wednesday
2  2020      2    8          1    15    Saturday
3  2020      2    3          1    15      Monday
4  2019     11   25          3    14      Monday


## Converting the new date columns to string

In [15]:
# Assuming train is your DataFrame with 'event_time' column already converted to datetime
train['year'] = train['event_time'].dt.year.astype(str)
train['month'] = train['event_time'].dt.month.astype(str)
train['day'] = train['event_time'].dt.day.astype(str)
train['hour'] = train['event_time'].dt.hour.astype(str)
train['day_range'] = train['day_range'].astype(str)

#dropping irrelevant columns
train.drop(columns='day', axis=1, inplace=True)
train = train.drop(columns=['session_start_time', 'session_expiry_time', 'event_time'])

## Removing the  User Screen Size Column

In [16]:
train.drop(columns='user_screen_size', inplace=True)

## Making the 'cart_total' value for the same session consistent with the checkout one for that session

In [17]:
# Assuming 'train' is your DataFrame

# Filter only the rows where 'page_type' is 'Checkout'
checkout_rows = train[train['page_type'] == 'Checkout'][['session_id', 'cart_total']]

# Merge the filtered rows back to the original DataFrame
train = train.merge(checkout_rows, on='session_id', suffixes=('', '_checkout'), how='left')

# Replace 'cart_total' with the value from 'cart_total_checkout' where available
train['cart_total'] = train['cart_total_checkout'].fillna(train['cart_total'])

# Drop the auxiliary column 'cart_total_checkout'
train.drop(columns=['cart_total_checkout'], inplace=True)

## TRIAL: Expanding the 'cart_data' Column:

In [18]:
# import ast
# import pandas as pd

# # Convert the string representation of dictionaries into actual dictionaries
# train['cart_data'] = train['cart_data'].apply(ast.literal_eval)

# # Explode the cart_data column to separate rows
# train = train.explode('cart_data').reset_index(drop=True)

# # Expand the cart_data dictionary into separate columns
# train = train.join(pd.json_normalize(train['cart_data']))

# # Drop the original cart_data column if needed
# train = train.drop(columns=['cart_data'])

In [19]:
# import numpy as np

# # Convert 'productPrice' and 'productUndiscountedPrice' columns to floats
# train['productPrice'] = train['productPrice'].astype(np.float64)
# train['productUndiscountedPrice'] = train['productUndiscountedPrice'].astype(np.float64)
# train['quantity'] = train['quantity'].astype(np.int64)

# # Check the data types after conversion
# print(train[['productPrice', 'productUndiscountedPrice']].dtypes)

In [20]:
# # Assuming 'train' is your DataFrame
# # Calculate average price for each category
# train['average_price'] = train.groupby('productCategory')['productPrice'].transform('mean')

# # Replace values in 'productPrice' column with average prices for each category
# train['productPrice'] = train['average_price']

# # Drop the 'average_price' column if not needed
# train.drop(columns=['average_price'], inplace = True)

In [21]:
# # Assuming 'train' is your DataFrame
# # Calculate average price for each category
# train['average_price'] = train.groupby('productCategory')['productUndiscountedPrice'].transform('mean')

# # Replace values in 'productPrice' column with average prices for each category
# train['productUndiscountedPrice'] = train['average_price']

# # Drop the 'average_price' column if not needed
# train.drop(columns=['average_price'], inplace = True)

In [22]:
# import pandas as pd

# # Assuming 'df' is your DataFrame with the 'productCategory' column
# # Calculate counts of each category
# category_counts = train['productCategory'].value_counts()

# # Create a DataFrame to store the category counts and their rankings
# category_ranking = pd.DataFrame({'category': category_counts.index, 'count': category_counts.values})

# # Rank the categories based on count
# category_ranking['rank'] = category_ranking['count'].rank(ascending=False).astype(int)

# # Merge the ranking DataFrame with the original DataFrame on the 'productCategory' column
# train = pd.merge(train, category_ranking, left_on='productCategory', right_on='category', how='left')

# # Drop the redundant 'category' and 'count' columns
# train.drop(['category', 'count'], axis=1, inplace=True)

# # Sort the DataFrame by the original index to restore the original order
# train.sort_index(inplace=True)

# # Display the DataFrame with the new 'rank' column
# train.head()

In [23]:
#train.drop(['productCategory'], axis=1, inplace=True

## Removing more Unecessecary Columns

In [24]:
train.drop(columns=['user_id','session_id', 'cart_data'], inplace=True)

## Determening the Numerical and the Categorical columns

In [25]:
numerical_cols = train.select_dtypes(include=['int64', 'float64']).columns
# Exclude the column named "total"
numerical_cols = numerical_cols[numerical_cols != 'total']
categorical_cols = train.select_dtypes(include=['object', 'category']).columns
print(numerical_cols)
print(categorical_cols)

Index(['cart_quantity', 'cart_total', 'last_reward_value', 'last_spend_value',
       'offer_display_count', 'duration_minutes'],
      dtype='object')
Index(['event_time_zone', 'event_type', 'page_type', 'user_status',
       'last_offer_type', 'offer_acceptance_state', 'year', 'month', 'hour',
       'day_of_week', 'day_range'],
      dtype='object')


## Normalizing the Numerical Columns

In [26]:
# Extract the numerical columns from the train DataFrame
numerical_df = train[numerical_cols]

# Convert integer variables to floats
numerical_df = numerical_df.astype(float)

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the numerical columns
numerical_df_normalized = pd.DataFrame(scaler.fit_transform(numerical_df), columns = numerical_cols)

## One Hot Encoding the Categorical Variables

In [27]:
# Step 1: Create a new DataFrame with selected categorical variables
categorical_df = train[categorical_cols].copy()

In [28]:
# Step 1: Initialize OneHotEncoder
encoder = OneHotEncoder()

categorical_df = categorical_df.astype(str)

# Step 2: Fit and transform the data
categorical_df_encoded = encoder.fit_transform(categorical_df)

# Step 3: Convert the encoded data to a DataFrame
categorical_df_encoded = pd.DataFrame(categorical_df_encoded.toarray(), columns=encoder.get_feature_names_out(categorical_df.columns))

#'categorical_df_encoded' is your new DataFrame with one-hot encoded categorical features

## Joining the Preprocessed Numerical and Categorical columns

In [29]:
final_df = pd.concat([numerical_df_normalized, train['total'], categorical_df_encoded], axis=1)

In [30]:
final_df.dropna(inplace=True)

In [31]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239702 entries, 0 to 239701
Data columns (total 90 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   cart_quantity                    239702 non-null  float64
 1   cart_total                       239702 non-null  float64
 2   last_reward_value                239702 non-null  float64
 3   last_spend_value                 239702 non-null  float64
 4   offer_display_count              239702 non-null  float64
 5   duration_minutes                 239702 non-null  float64
 6   total                            239702 non-null  float64
 7   event_time_zone_-120             239702 non-null  float64
 8   event_time_zone_-180             239702 non-null  float64
 9   event_time_zone_-240             239702 non-null  float64
 10  event_time_zone_-300             239702 non-null  float64
 11  event_time_zone_-330             239702 non-null  float64
 12  ev

# Preprocessing and Feature Engineering for the Spitted Test Set

## Removing Zero or Near Zero Columns

In [32]:
valid = valid.reset_index(drop=True)
valid = valid.drop('offer_decline_count', axis=1)

In [33]:
valid.dropna(thresh=valid.shape[1]-1, inplace=True)
valid.reset_index(drop=True, inplace=True)

## Adding the Duration of each Session

In [34]:
# Group by 'session_id' and calculate the duration
session_durations_valid = valid.groupby('session_id').agg({
    'session_start_time': 'min',
    'session_expiry_time': 'max'
}).reset_index()

# Calculate the duration as the difference between the max expiry time and the min start time
session_durations_valid['duration'] = session_durations_valid['session_expiry_time'] - session_durations_valid['session_start_time']

# Convert duration to minutes
session_durations_valid['duration_minutes'] = session_durations_valid['duration'].dt.total_seconds() / 60
session_durations_valid['duration_minutes'] = session_durations_valid['duration_minutes'].round(2)


# Merge back into the original dataframe
valid = valid.merge(session_durations_valid[['session_id', 'duration_minutes']], on='session_id', how='left')

## Extracting the year, month, day, hour, and day name from session_start_time

In [35]:
# Extract year, month, day, hour, and day name from event_time before normalizing
valid['year'] = valid['event_time'].dt.year.astype(str)
valid['month'] = valid['event_time'].dt.month.astype(str)
valid['day'] = valid['event_time'].dt.day
valid['hour'] = valid['event_time'].dt.hour.astype(str)
valid['day_of_week'] = valid['event_time'].dt.day_name()

# Now you have separate columns for year, month, day, hour, and day_of_week along with a combined string if needed
print(valid[['year', 'month', 'day', 'hour', 'day_of_week']].head())

   year month  day hour day_of_week
0  2020     3    4    6   Wednesday
1  2020     2    3   14      Monday
2  2019    11   20   15   Wednesday
3  2020     1    4    5    Saturday
4  2019    11   11    5      Monday


## Adding a Day Range based on the New 'day' column

In [36]:
# Function to categorize days with updated labels
def categorize_day(day):
    if 1 <= day <= 10:
        return 1  # Updated label for days 1-10
    elif 11 <= day <= 21:
        return 2  # Updated label for days 11-21
    else:
        return 3  # Updated label for days 22-31

# Apply the updated function to the 'day' column to create a new 'day_range' column with numerical labels
valid['day_range'] = valid['day'].apply(categorize_day)


## Converting the new date columns to string

In [37]:
# Assuming valid is your DataFrame with 'event_time' column already converted to datetime
valid['year'] = valid['event_time'].dt.year.astype(str)
valid['month'] = valid['event_time'].dt.month.astype(str)
valid['day'] = valid['event_time'].dt.day.astype(str)
valid['hour'] = valid['event_time'].dt.hour.astype(str)
valid['day_range'] = valid['day_range'].astype(str)

valid.drop(columns=['day'], axis = 1, inplace=True)

valid = valid.drop(columns=['session_start_time', 'session_expiry_time', 'event_time'])

## Removing the  User Screen Size Column

In [38]:
valid.drop(columns='user_screen_size', inplace=True)

## Making the 'cart_total' value for the same session consistent with the checkout one for that session

In [39]:
# Assuming 'train' is your DataFrame

# Filter only the rows where 'page_type' is 'Checkout'
checkout_rows = valid[valid['page_type'] == 'Checkout'][['session_id', 'cart_total']]

# Merge the filtered rows back to the original DataFrame
valid = valid.merge(checkout_rows, on='session_id', suffixes=('', '_checkout'), how='left')

# Replace 'cart_total' with the value from 'cart_total_checkout' where available
valid['cart_total'] = valid['cart_total_checkout'].fillna(valid['cart_total'])

# Drop the auxiliary column 'cart_total_checkout'
valid.drop(columns=['cart_total_checkout'], inplace=True)

## TRIAL: Expanding the 'cart_data' Column

In [40]:
# import ast
# import pandas as pd

# # Convert the string representation of dictionaries into actual dictionaries
# valid['cart_data'] = valid['cart_data'].apply(ast.literal_eval)

# # Explode the cart_data column to separate rows
# valid = valid.explode('cart_data').reset_index(drop=True)

# # Expand the cart_data dictionary into separate columns
# valid = valid.join(pd.json_normalize(valid['cart_data']))

# # Drop the original cart_data column if needed
# valid = valid.drop(columns=['cart_data'])

In [41]:
# import numpy as np

# # Convert 'productPrice' and 'productUndiscountedPrice' columns to floats
# valid['productPrice'] = valid['productPrice'].astype(np.float64)
# valid['productUndiscountedPrice'] = valid['productUndiscountedPrice'].astype(np.float64)
# valid['quantity'] = valid['quantity'].astype(np.int64)

# # Check the data types after conversion
# print(valid[['productPrice', 'productUndiscountedPrice']].dtypes)

In [42]:
# # Assuming 'valid' is your DataFrame
# # Calculate average price for each category
# valid['average_price'] = valid.groupby('productCategory')['productPrice'].transform('mean')

# # Replace values in 'productPrice' column with average prices for each category
# valid['productPrice'] = valid['average_price']

# # Drop the 'average_price' column if not needed
# valid.drop(columns=['average_price'], inplace=True

In [43]:
# # Assuming 'valid' is your DataFrame
# # Calculate average price for each category
# valid['average_price'] = valid.groupby('productCategory')['productUndiscountedPrice'].transform('mean')

# # Replace values in 'productPrice' column with average prices for each category
# valid['productUndiscountedPrice'] = valid['average_price']

# # Drop the 'average_price' column if not needed
# valid.drop(columns=['average_price'], inplace=True)

In [44]:
# import pandas as pd

# # Assuming 'valid' is your DataFrame with the 'productCategory' column
# # Calculate counts of each category
# category_counts = valid['productCategory'].value_counts()

# # Create a DataFrame to store the category counts and their rankings
# category_ranking = pd.DataFrame({'category': category_counts.index, 'count': category_counts.values})

# # Rank the categories based on count
# category_ranking['rank'] = category_ranking['count'].rank(ascending=False).astype(int)

# # Merge the ranking DataFrame with the original DataFrame on the 'productCategory' column
# valid = pd.merge(valid, category_ranking, left_on='productCategory', right_on='category', how='left')

# # Drop the redundant 'category' and 'count' columns
# valid.drop(['category', 'count'], axis=1, inplace=True)

# # Sort the DataFrame by the original index to restore the original order
# valid.sort_index(inplace=True)

# # Display the DataFrame with the new 'rank' column
# valid.head()

In [45]:
#valid.drop(['productCategory'], axis=1, inplace=True)

## Removing more Unecessecary Columns

In [46]:
valid.drop(columns=['user_id','session_id', 'cart_data'], inplace=True)

## Determening the Numerical and the Categorical columns

In [47]:
numerical_cols_valid = valid.select_dtypes(include=['int64', 'float64']).columns
# Exclude the column named "total"
numerical_cols_valid = numerical_cols_valid[numerical_cols_valid != 'total']
categorical_cols_valid = valid.select_dtypes(include=['object', 'category']).columns

print(numerical_cols_valid)
print(categorical_cols_valid)

Index(['cart_quantity', 'cart_total', 'last_reward_value', 'last_spend_value',
       'offer_display_count', 'duration_minutes'],
      dtype='object')
Index(['event_time_zone', 'event_type', 'page_type', 'user_status',
       'last_offer_type', 'offer_acceptance_state', 'year', 'month', 'hour',
       'day_of_week', 'day_range'],
      dtype='object')


## Normalizing the Numerical Columns

In [48]:
# Extract the numerical columns from the valid DataFrame
numerical_df_valid = valid[numerical_cols_valid].copy()

# Convert integer variables to floats if necessary
numerical_df_valid = numerical_df_valid.astype(float)

# Transform the numerical columns using the fitted scaler
numerical_df_normalized_valid = pd.DataFrame(scaler.transform(numerical_df_valid), columns=numerical_cols_valid)

## One Hot Encoding the Categorical Variables

In [49]:
# Step 1: Create a new DataFrame with selected categorical variables
categorical_df_valid = valid[categorical_cols].copy()

In [50]:
# Assuming 'categorical_df' is your DataFrame with only categorical features

# Step 1: Initialize OneHotEncoder
encoder = OneHotEncoder()

# Step 2: Fit and transform the data
categorical_df_encoded_valid = encoder.fit_transform(categorical_df_valid)

# Step 3: Convert the encoded data to a DataFrame
categorical_df_encoded_valid = pd.DataFrame(categorical_df_encoded_valid.toarray(), columns=encoder.get_feature_names_out(categorical_df_valid.columns))

# 'categorical_df_encoded' is your new DataFrame with one-hot encoded categorical features

## Joining the Preprocessed Numerical and Categorical columns

In [51]:
final_df_valid = pd.concat([numerical_df_normalized_valid,valid['total'], categorical_df_encoded_valid], axis=1)

In [52]:
final_df_valid.dropna(inplace=True)

In [53]:
final_df.head()

,cart_quantity,cart_total,last_reward_value,last_spend_value,offer_display_count,duration_minutes,total,event_time_zone_-120,event_time_zone_-180,event_time_zone_-240,...,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday,day_range_1,day_range_2,day_range_3
0,0.008929,0.001178,0.0,0.012821,0.000000,0.071951,96.67,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,0.008929,0.001805,0.0,0.000000,0.000000,0.059829,49.33,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,0.008929,0.001910,0.0,0.025641,0.000000,0.007153,104.98,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.035714,0.003282,0.0,0.025641,0.015873,0.049920,99.98,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.008929,0.001075,0.0,0.000000,0.000000,0.055341,59.98,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


# Models

## Splitting the Data Set into X_train, y_train, X_test, y_test

In [54]:
X_train = final_df.drop(columns=['total'])  # Selecting all columns except 'total'
y_train = final_df['total']
X_test=final_df_valid.drop(columns=['total'])
y_test= final_df_valid['total']

## Fixing Potential Columns Discrepencie between X_train and X_test

In [55]:
# Get the columns of X_train and X_test
train_columns = set(X_train.columns)
test_columns = set(X_test.columns)

# Columns present in X_train but not in X_test
missing_in_test = train_columns - test_columns

# Columns present in X_test but not in X_train
missing_in_train = test_columns - train_columns

print("Columns present in X_train but not in X_test:", missing_in_test)
print("Columns present in X_test but not in X_train:", missing_in_train)

Columns present in X_train but not in X_test: {'event_time_zone_-840', 'event_time_zone_-420', 'hour_22', 'event_time_zone_-780', 'event_time_zone_-240', 'event_time_zone_-660'}
Columns present in X_test but not in X_train: {'event_time_zone_-210'}


In [56]:
# Assuming X_train is your DataFrame and missing_in_test is your list of column names to drop
X_train.drop(columns=missing_in_test, inplace=True)
X_test.drop(columns=missing_in_train, inplace=True)

## Neural Network Model

In [57]:
# Define the neural network architecture
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(35, activation='relu'),
    Dense(1)  # Output layer with one neuron for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=116, batch_size=850, validation_data=(X_test, y_test), verbose=0)

# Predict on training and testing data
train_predictions = model.predict(X_train).flatten()
test_predictions = model.predict(X_test).flatten()

# Compute RMSE
train_rmse = np.sqrt(mean_squared_error(y_train, train_predictions))
test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))

print("Training RMSE:", train_rmse)
print("Testing RMSE:", test_rmse)


c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7491/7491 ━━━━━━━━━━━━━━━━━━━━ 7s 888us/step
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 1s 718us/step
Training RMSE: 439.1727834785868
Testing RMSE: 195.13961976756286


In [58]:
y_test.mean()

218.6267556315644

## TRIAL: Different Models

In [59]:
# from sklearn.linear_model import LinearRegression

# # Train your model (Linear Regression)
# model = LinearRegression()
# model.fit(X_train_pca, y_train)

# # Predictions
# train_predictions = model.predict(X_train_pca)
# test_predictions = model.predict(X_test_pca)

# # Compute Mean Squared Error (MSE)
# train_mse = mean_squared_error(y_train, train_predictions)
# test_mse = mean_squared_error(y_test, test_predictions)

# # Compute Root Mean Squared Error (RMSE)
# train_rmse = np.sqrt(train_mse)
# test_rmse = np.sqrt(test_mse)

# print("Training RMSE (Linear Regression):", train_rmse)
# print("Testing RMSE (Linear Regression):", test_rmse)

In [60]:
# from sklearn.tree import DecisionTreeRegressor

# # Adjusting the Decision Tree parameters
# model = DecisionTreeRegressor(
#     random_state=42,
#     max_depth=8,  # Example: limit the depth of the tree
#     min_samples_split=15,  # Example: require at least 10 samples to split a node
#     min_samples_leaf=8  # Example: require at least 5 samples at each leaf
# )

# # Train the model
# model.fit(X_train_pca, y_train)

# # Predictions
# train_predictions = model.predict(X_train_pca)
# test_predictions = model.predict(X_test_pca)

# # Compute Mean Squared Error (MSE) and Root Mean Squared Error (RMSE)
# train_mse = mean_squared_error(y_train, train_predictions)
# test_mse = mean_squared_error(y_test, test_predictions)
# train_rmse = np.sqrt(train_mse)
# test_rmse = np.sqrt(test_mse)

# print("Training RMSE:", train_rmse)
# print("Testing RMSE:", test_rmse)

In [61]:
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.metrics import mean_squared_error
# import numpy as np

# # Adjusting the Decision Tree parameters
# model = DecisionTreeRegressor(
#     random_state=42,
#     max_depth=8,  # Limit the depth of the tree
#     min_samples_split=13,  # Require at least 20 samples to split a node
#     min_samples_leaf=5  # Require at least 5 samples at each leaf
# )

# # Train the model on the original data
# model.fit(X_train, y_train)

# # Predictions on the original data
# train_predictions = model.predict(X_train)
# test_predictions = model.predict(X_test)

# # Compute Mean Squared Error (MSE) and Root Mean Squared Error (RMSE)
# train_mse = mean_squared_error(y_train, train_predictions)
# test_mse = mean_squared_error(y_test, test_predictions)
# train_rmse = np.sqrt(train_mse)
# test_rmse = np.sqrt(test_mse)

# print("Training RMSE:", train_rmse)
# print("Testing RMSE:", test_rmse)

In [62]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error
# import numpy as np

# # Setting up the Random Forest Regressor
# rf_model = RandomForestRegressor(
#     random_state=42,
#     n_estimators=90,  # Number of trees in the forest
#     max_depth=8,  # Limit the depth of each tree to prevent overfitting
#     min_samples_split=15,  # Minimum number of samples required to split an internal node
#     min_samples_leaf=5  # Minimum number of samples required to be at a leaf node
# )

# # Training the model on the original training data
# rf_model.fit(X_train, y_train)

# # Making predictions on both training and testing data
# rf_train_predictions = rf_model.predict(X_train)
# rf_test_predictions = rf_model.predict(X_test)

# # Calculating Mean Squared Error (MSE) and Root Mean Squared Error (RMSE)
# rf_train_mse = mean_squared_error(y_train, rf_train_predictions)
# rf_test_mse = mean_squared_error(y_test, rf_test_predictions)
# rf_train_rmse = np.sqrt(rf_train_mse)
# rf_test_rmse = np.sqrt(rf_test_mse)

# # Outputting the RMSE results
# print("Random Forest Training RMSE:", rf_train_rmse)
# print("Random Forest Testing RMSE:", rf_test_rmse)

In [63]:
# #Define the KNN model
# knn = KNeighborsRegressor()

# #Define the grid of hyperparameters to search
# param_grid = {
#    'n_neighbors': [3, 5, 7, 9],  # Number of neighbors to consider
#    'weights': ['uniform', 'distance'],  # Weight function used in prediction
#    'p': [1, 2]  # Power parameter for the Minkowski metric
# }

# #Perform grid search cross-validation
# grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
# grid_search.fit(X_train, y_train)

# #Get the best hyperparameters
# best_params = grid_search.best_params_
# print("Best Hyperparameters:", best_params)

# #Use the best model to make predictions
# best_model = grid_search.best_estimator_
# y_pred_train = best_model.predict(X_train)
# y_pred_test = best_model.predict(X_test)

# #Calculate RMSE on training and test data
# rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
# rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
# print("RMSE on training data:", rmse_train)
# print("RMSE on test data:", rmse_test)

In [64]:
# from sklearn.neighbors import KNeighborsRegressor

# # Define the KNN model with a specific number of neighbors
# knn = KNeighborsRegressor(n_neighbors=30)  # You can change the value of n_neighbors as per your choice

# # Train the model
# knn.fit(X_train, y_train)

# # Make predictions
# y_pred_train = knn.predict(X_train)
# y_pred_test = knn.predict(X_test)

# # Calculate RMSE on training and test data
# rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
# rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
# print("RMSE on training data:", rmse_train)
# print("RMSE on test data:", rmse_test)

# Preprocessing and Feature Engineering for the Spitted Test Set

## Reading the Kaggle Test Set

In [65]:
data_kaggle = pd.read_csv("C:\\Users\\Lenovo\\OneDrive\\01_Education\\02_AUB\\02_Spring2024\\MSBA315-Introduction to Python for Machine Learning\\Assignments\\Assignment5\\AssignmentFolder\\data\\test.csv")

## Reading the Train Set to Retrain the Model

In [66]:
data_train_kaggle = pd.read_csv('C:\\Users\\Lenovo\\OneDrive\\01_Education\\02_AUB\\02_Spring2024\\MSBA315-Introduction to Python for Machine Learning\\Assignments\\Assignment5\\AssignmentFolder\\data\\data.csv')

In [67]:
data_kaggle.head()

,ID,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13,col14,col15,col16,col17,col18,col19
0,0,785cc174-f549-4528-9ddf-6cbbd379212e,6777e217-dca4-4443-b60f-91ca4cf08136,2019-08-04T03:24:37.048Z,2019-08-04T04:29:38.875Z,2019-08-04T03:59:59.963Z,180,PAGE_LOAD,ProductDetailPage,0,PBu,7,307.61,"[{'productPrice': '44.99', 'productCategory': ...",C,5.0,125.0,1.0,3.145728e+06,IGNORED
1,1,b0d0ee4e-9472-467e-91bc-bdf77f8a84c0,a4260259-961a-4ab1-9dd6-50a9f0c05079,2019-08-04T03:32:40.740Z,2019-08-04T04:29:42.477Z,2019-08-04T04:00:05.579Z,240,CART_CHANGE,Cart,0,PBu,5,349.62,"[{'productPrice': '39.99', 'productCategory': ...",P,5.0,50.0,1.0,1.049088e+06,IGNORED
2,2,8fa04b12-e187-405c-a30e-b27f677ed12f,4f723b51-a83d-4863-860d-f3f3419fcade,2019-08-04T03:55:34.356Z,2019-08-04T04:29:15.550Z,2019-08-04T04:00:06.161Z,420,PAGE_LOAD,Checkout,0,PBr,1,219.99,"[{'productPrice': '219.99', 'productCategory':...",C,10.0,100.0,1.0,3.145728e+06,IGNORED
3,3,304824f0-0556-4fda-a616-a27a1ad31dad,6b6e9f89-53c9-4005-9b60-a6e23c01c90f,2019-08-04T03:16:58.809Z,2019-08-04T04:29:36.673Z,2019-08-04T04:00:13.654Z,420,PAGE_LOAD,ProductSubCategory,0,PBr,1,39.99,"[{'productPrice': '39.99', 'productCategory': ...",S,5.0,50.0,1.0,2.592000e+06,IGNORED
4,4,3c09349d-a803-4663-8dc3-2eb93732bb4e,92909fbd-b918-4b9b-aba8-a1d67493ba6f,2019-08-04T03:03:04.102Z,2019-08-04T04:29:04.603Z,2019-08-04T04:00:16.768Z,240,PAGE_LOAD,ProductSubCategory,0,PBu,2,79.32,"[{'productPrice': '39.33', 'productCategory': ...",C,5.0,50.0,1.0,1.024634e+06,IGNORED


## Fetching the Column Names of the Train Dataset

In [68]:
column_names = data_train_kaggle.columns.tolist()
column_names = column_names[:-1]
column_names

['user_id',
 'session_id',
 'session_start_time',
 'session_expiry_time',
 'event_time',
 'event_time_zone',
 'event_type',
 'page_type',
 'offer_decline_count',
 'user_status',
 'cart_quantity',
 'cart_total',
 'cart_data',
 'last_offer_type',
 'last_reward_value',
 'last_spend_value',
 'offer_display_count',
 'user_screen_size',
 'offer_acceptance_state']

## Seperating the ID and the rest of the Features in the Kaggle Dataset

In [69]:
import pandas as pd

# Create a new DataFrame with only the first column
first_column_df = data_kaggle.iloc[:, :1]

# Create a new DataFrame with all columns except the first column
new_df =data_kaggle.iloc[:, 1:]



## Replacing the Column Names on the Kaggle Dataset 

In [70]:
# Rename columns of the DataFrame
new_df.columns = column_names

In [71]:
new_df.head()

,user_id,session_id,session_start_time,session_expiry_time,event_time,event_time_zone,event_type,page_type,offer_decline_count,user_status,cart_quantity,cart_total,cart_data,last_offer_type,last_reward_value,last_spend_value,offer_display_count,user_screen_size,offer_acceptance_state
0,785cc174-f549-4528-9ddf-6cbbd379212e,6777e217-dca4-4443-b60f-91ca4cf08136,2019-08-04T03:24:37.048Z,2019-08-04T04:29:38.875Z,2019-08-04T03:59:59.963Z,180,PAGE_LOAD,ProductDetailPage,0,PBu,7,307.61,"[{'productPrice': '44.99', 'productCategory': ...",C,5.0,125.0,1.0,3.145728e+06,IGNORED
1,b0d0ee4e-9472-467e-91bc-bdf77f8a84c0,a4260259-961a-4ab1-9dd6-50a9f0c05079,2019-08-04T03:32:40.740Z,2019-08-04T04:29:42.477Z,2019-08-04T04:00:05.579Z,240,CART_CHANGE,Cart,0,PBu,5,349.62,"[{'productPrice': '39.99', 'productCategory': ...",P,5.0,50.0,1.0,1.049088e+06,IGNORED
2,8fa04b12-e187-405c-a30e-b27f677ed12f,4f723b51-a83d-4863-860d-f3f3419fcade,2019-08-04T03:55:34.356Z,2019-08-04T04:29:15.550Z,2019-08-04T04:00:06.161Z,420,PAGE_LOAD,Checkout,0,PBr,1,219.99,"[{'productPrice': '219.99', 'productCategory':...",C,10.0,100.0,1.0,3.145728e+06,IGNORED
3,304824f0-0556-4fda-a616-a27a1ad31dad,6b6e9f89-53c9-4005-9b60-a6e23c01c90f,2019-08-04T03:16:58.809Z,2019-08-04T04:29:36.673Z,2019-08-04T04:00:13.654Z,420,PAGE_LOAD,ProductSubCategory,0,PBr,1,39.99,"[{'productPrice': '39.99', 'productCategory': ...",S,5.0,50.0,1.0,2.592000e+06,IGNORED
4,3c09349d-a803-4663-8dc3-2eb93732bb4e,92909fbd-b918-4b9b-aba8-a1d67493ba6f,2019-08-04T03:03:04.102Z,2019-08-04T04:29:04.603Z,2019-08-04T04:00:16.768Z,240,PAGE_LOAD,ProductSubCategory,0,PBu,2,79.32,"[{'productPrice': '39.33', 'productCategory': ...",C,5.0,50.0,1.0,1.024634e+06,IGNORED


## Rejoining the ID column with the Kaggle Dataset

In [72]:
# Merge the two DataFrames on index
data_kaggle = pd.concat([first_column_df, new_df], axis=1)

In [73]:
data_kaggle.shape

(875707, 20)

## Converting Time Columns to DateTime Objects

In [74]:
# Convert object columns to datetime objects
data_kaggle['session_start_time'] = pd.to_datetime(data_kaggle['session_start_time'])
data_kaggle['session_expiry_time'] = pd.to_datetime(data_kaggle['session_expiry_time'])
data_kaggle['event_time'] = pd.to_datetime(data_kaggle['event_time'])
data_kaggle['event_time_zone'] = data_kaggle['event_time_zone'].astype('str')

## Removing Zero or Near Zero Columns

In [75]:
#mean is approx to 0 in 'offer_decline_count"
data_kaggle = data_kaggle.drop('offer_decline_count', axis=1)
data_kaggle = data_kaggle.reset_index(drop=True)

## Adding the Duration of each Session

In [76]:
# Group by 'session_id' and calculate the duration
session_durations = data_kaggle.groupby('session_id').agg({
    'session_start_time': 'min',
    'session_expiry_time': 'max'
}).reset_index()

# Calculate the duration as the difference between the max expiry time and the min start time
session_durations['duration'] = session_durations['session_expiry_time'] - session_durations['session_start_time']

# Convert duration to a more convenient unit: minutes
session_durations['duration_minutes'] = session_durations['duration'].dt.total_seconds() / 60  # Convert seconds to minutes

# Round the duration_minutes column to 2 decimal places (nearest 0.01)
session_durations['duration_minutes'] = session_durations['duration_minutes'].round(2)

# Now merge this back into the original dataframe
data_kaggle = data_kaggle.merge(session_durations[['session_id', 'duration_minutes']], on='session_id', how='left')

# Verify the results
print(data_kaggle[['session_id', 'session_start_time', 'session_expiry_time', 'duration_minutes']].head())

                             session_id               session_start_time  \
0  6777e217-dca4-4443-b60f-91ca4cf08136 2019-08-04 03:24:37.048000+00:00   
1  a4260259-961a-4ab1-9dd6-50a9f0c05079 2019-08-04 03:32:40.740000+00:00   
2  4f723b51-a83d-4863-860d-f3f3419fcade 2019-08-04 03:55:34.356000+00:00   
3  6b6e9f89-53c9-4005-9b60-a6e23c01c90f 2019-08-04 03:16:58.809000+00:00   
4  92909fbd-b918-4b9b-aba8-a1d67493ba6f 2019-08-04 03:03:04.102000+00:00   

               session_expiry_time  duration_minutes  
0 2019-08-04 04:29:38.875000+00:00             82.08  
1 2019-08-04 04:29:42.477000+00:00             86.12  
2 2019-08-04 04:29:15.550000+00:00             38.64  
3 2019-08-04 04:29:36.673000+00:00             90.64  
4 2019-08-04 04:29:04.603000+00:00            119.72  


## Extracting the year, month, day, hour, and day name from session_start_time

In [77]:
# Extract year, month, day, hour, and day name from session_start_time before normalizing
data_kaggle['year'] = data_kaggle['event_time'].dt.year
data_kaggle['month'] = data_kaggle['event_time'].dt.month
data_kaggle['day'] = data_kaggle['event_time'].dt.day
data_kaggle['hour'] = data_kaggle['event_time'].dt.hour  # Extract hour before normalizing
data_kaggle['day_of_week'] = data_kaggle['event_time'].dt.day_name()

# Now you have separate columns for year, month, day, hour, and day_of_week along with a combined string if needed
print(data_kaggle[['year', 'month', 'day', 'hour', 'day_of_week']].head())

   year  month  day  hour day_of_week
0  2019      8    4     3      Sunday
1  2019      8    4     4      Sunday
2  2019      8    4     4      Sunday
3  2019      8    4     4      Sunday
4  2019      8    4     4      Sunday


## Adding a Day Range based on the New 'day' column

In [78]:
# Function to categorize days with updated labels
def categorize_day(day):
    if 1 <= day <= 10:
        return 1  # Updated label for days 1-10
    elif 11 <= day <= 21:
        return 2  # Updated label for days 11-21
    else:
        return 3  # Updated label for days 22-31

# Apply the updated function to the 'day' column to create a new 'day_range' column with numerical labels
data_kaggle['day_range'] = data_kaggle['day'].apply(categorize_day)

# Display the first few rows to verify the new column
print(data_kaggle[['year', 'month', 'day', 'day_range', 'hour', 'day_of_week']].head())

   year  month  day  day_range  hour day_of_week
0  2019      8    4          1     3      Sunday
1  2019      8    4          1     4      Sunday
2  2019      8    4          1     4      Sunday
3  2019      8    4          1     4      Sunday
4  2019      8    4          1     4      Sunday


## Converting the new date columns to string

In [79]:
# Assuming train is your DataFrame with 'event_time' column already converted to datetime
data_kaggle['year'] = data_kaggle['event_time'].dt.year.astype(str)
data_kaggle['month'] = data_kaggle['event_time'].dt.month.astype(str)
data_kaggle['day'] = data_kaggle['event_time'].dt.day.astype(str)
data_kaggle['hour'] = data_kaggle['event_time'].dt.hour.astype(str)
data_kaggle['day_range'] = data_kaggle['day_range'].astype(str)

#dropping irrelevant columns
data_kaggle.drop(columns='day', axis=1, inplace=True)
data_kaggle = data_kaggle.drop(columns=['session_start_time', 'session_expiry_time', 'event_time'])

## Removing the  User Screen Size Column

In [80]:
data_kaggle.drop(columns='user_screen_size', inplace=True)

## Making the 'cart_total' value for the same session consistent with the checkout one for that session

In [81]:
import pandas as pd

# Assuming 'train' is your DataFrame

# Filter only the rows where 'page_type' is 'Checkout'
checkout_rows = data_kaggle[data_kaggle['page_type'] == 'Checkout'][['session_id', 'cart_total']]

# Group by 'session_id' and aggregate the 'cart_total' column
checkout_rows = checkout_rows.groupby('session_id').first().reset_index()

# Merge the filtered rows back to the original DataFrame
data_kaggle = data_kaggle.merge(checkout_rows, on='session_id', suffixes=('', '_checkout'), how='left')

# Replace 'cart_total' with the value from 'cart_total_checkout' where available
data_kaggle['cart_total'] = data_kaggle['cart_total_checkout'].fillna(data_kaggle['cart_total'])

# Drop the auxiliary column 'cart_total_checkout'
data_kaggle.drop(columns=['cart_total_checkout'], inplace=True)


## Removing more Unecessecary Columns

In [82]:
data_kaggle.drop(columns=['user_id','session_id', 'cart_data'], inplace=True)

## Determening the Numerical and the Categorical columns

In [83]:
numerical_cols_kaggle = train.select_dtypes(include=['int64', 'float64']).columns
# Exclude the column named "total"
#numerical_cols_kaggle = numerical_cols_kaggle[numerical_cols != 'total']
categorical_cols_kaggle = train.select_dtypes(include=['object', 'category']).columns
print(numerical_cols_kaggle)
print(categorical_cols_kaggle)

Index(['cart_quantity', 'cart_total', 'last_reward_value', 'last_spend_value',
       'offer_display_count', 'total', 'duration_minutes'],
      dtype='object')
Index(['event_time_zone', 'event_type', 'page_type', 'user_status',
       'last_offer_type', 'offer_acceptance_state', 'year', 'month', 'hour',
       'day_of_week', 'day_range'],
      dtype='object')


## Normalizing the Numerical Columns

In [84]:
# Extract the numerical columns from the valid DataFrame
numerical_df_valid = data_kaggle[numerical_cols_valid].copy()

# Convert integer variables to floats if necessary
numerical_df_valid = numerical_df_valid.astype(float)

# Transform the numerical columns using the fitted scaler
numerical_df_normalized_valid = pd.DataFrame(scaler.transform(numerical_df_valid), columns=numerical_cols_valid)

## One Hot Encoding the Categorical Variables

In [85]:
# Step 1: Create a new DataFrame with selected categorical variables
categorical_df_valid = data_kaggle[categorical_cols].copy()

In [86]:
# Assuming 'categorical_df' is your DataFrame with only categorical features

# Step 1: Initialize OneHotEncoder
encoder = OneHotEncoder()

# Step 2: Fit and transform the data
categorical_df_encoded_valid = encoder.fit_transform(categorical_df_valid)

# Step 3: Convert the encoded data to a DataFrame
categorical_df_encoded_valid = pd.DataFrame(categorical_df_encoded_valid.toarray(), columns=encoder.get_feature_names_out(categorical_df_valid.columns))

# 'categorical_df_encoded' is your new DataFrame with one-hot encoded categorical features

## Joining the Preprocessed Numerical and Categorical columns

In [87]:
data_kaggle = pd.concat([numerical_df_normalized_valid, categorical_df_encoded_valid], axis=1)

## Removing the ID column from the Dataset to Train the Model

In [88]:
import pandas as pd

# Create a new DataFrame with only the first column
first_column_df = data_kaggle.iloc[:, :1]

# Create a new DataFrame with all columns except the first column
data_kaggle_test =data_kaggle.iloc[:, 1:]

## Fixing Potential Columns Discrepencie between X_train and X_test

In [89]:
# Get the columns of X_train and X_test
train_columns = set(X_train.columns)
test_columns = set(data_kaggle_test.columns)

# Columns present in X_train but not in X_test
missing_in_test = train_columns - test_columns

# Columns present in X_test but not in X_train
missing_in_train = test_columns - train_columns

print("Columns present in X_train but not in X_test:", missing_in_test)
print("Columns present in X_test but not in X_train:", missing_in_train)

Columns present in X_train but not in X_test: {'event_time_zone_60', 'month_12', 'hour_20', 'month_11', 'month_10', 'month_9', 'event_time_zone_540', 'cart_quantity', 'month_2', 'event_time_zone_-630', 'month_1'}
Columns present in X_test but not in X_train: {'hour_0', 'event_time_zone_-570', 'month_4', 'hour_23', 'offer_acceptance_state_nan', 'month_8', 'event_time_zone_-420', 'hour_22', 'event_time_zone_-780', 'event_time_zone_-240', 'event_time_zone_-600', 'hour_21', 'month_7', 'last_offer_type_nan'}


In [90]:
# Assuming X_train is your DataFrame and missing_in_test is your list of column names to drop
X_train.drop(columns=missing_in_test, inplace=True)
data_kaggle_test.drop(columns=missing_in_train, inplace=True)

# Best Kaggle model

## Neural Network Model (Best)

In [91]:
# Define the neural network architecture
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(35, activation='relu'),
    Dense(1)  # Output layer with one neuron for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=116, batch_size=850, verbose=0)

# Predict on training and testing data
train_predictions = model.predict(X_train).flatten()
test_predictions = model.predict(data_kaggle_test).flatten()

# Compute RMSE
train_rmse = np.sqrt(mean_squared_error(y_train, train_predictions))

print("Training RMSE:", train_rmse)

# Create a DataFrame with the test predictions
test_predictions_df = pd.DataFrame({'test_predictions': test_predictions})

print(test_predictions_df.head())  # Display the first few rows of the DataFrame

c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7491/7491 ━━━━━━━━━━━━━━━━━━━━ 7s 930us/step
27366/27366 ━━━━━━━━━━━━━━━━━━━━ 26s 944us/step
Training RMSE: 443.85792919125004
   test_predictions
0        302.731964
1         87.540878
2        163.611389
3         83.827530
4        112.715942


# Converting the Prediciton Dataset to CSV

In [92]:
test_predictions_df.head()

,test_predictions
0,302.731964
1,87.540878
2,163.611389
3,83.827530
4,112.715942


In [93]:
data_kaggle_ID = pd.read_csv("C:\\Users\\Lenovo\\OneDrive\\01_Education\\02_AUB\\02_Spring2024\\MSBA315-Introduction to Python for Machine Learning\\Assignments\\Assignment5\\AssignmentFolder\\data\\test.csv")

In [94]:
# Keep only the first column
data_kaggle_ID = data_kaggle_ID.iloc[:, [0]]

In [95]:
#test= pd.read_csv('test.csv')

# Create a DataFrame with the IDs and predictions
result_df = pd.DataFrame({
    'ID': data_kaggle_ID['ID'],  # Assuming 'id' is a column in X_test
    'predicted_amount': test_predictions_df['test_predictions']
})

# Save the DataFrame to a CSV file
#result_df.to_csv('predictions.csv', index=False)

In [96]:
result_df.head()

,ID,predicted_amount
0,0,302.731964
1,1,87.540878
2,2,163.611389
3,3,83.827530
4,4,112.715942


In [97]:
#Save the DataFrame to a CSV file
result_df.to_csv('C:\\Users\\Lenovo\\Desktop\\predictions.csv', index=False)

In [98]:
#result_df.to_csv('C:\\Users\\Lenovo\\OneDrive\\01_Education\\02_AUB\\02_Spring2024\\MSBA315-Introduction to Python for Machine Learning\\Assignments\\Assignment5\\AssignmentFolder\\data\\predictions.csv', index=False)